In diesem Notebook möchte ich die autograd funktion von pytorch anhand eines kleinen Netzwerkes testen.

In [59]:
import numpy as np
import torch
import torch.nn as nn

Erstellung des einfachen Netzwerkes

In [60]:
class SimpleNN(nn.Module):
    def __init__(self):
        super(SimpleNN, self).__init__()
        # Schichten festlegen
        self.Layer_1 = nn.Linear(3, 16)
        self.Output_Layer = nn.Linear(16, 2)

    def forward(self, x):
        x = self.Layer_1(x)
        x = self.Output_Layer(x)

        return x

test_network = SimpleNN()

Ein paar einfache werte austesten (mit Gradientenverfolgung des Eingangs)

In [61]:
test_input = torch.tensor([1, 2, 1.5], dtype=torch.float32, requires_grad=True)
print('Test Input: \n', test_input)
test_output = test_network(test_input)

print('Ergebnis: \n', test_output)

Test Input: 
 tensor([1.0000, 2.0000, 1.5000], requires_grad=True)
Ergebnis: 
 tensor([0.4049, 0.2036], grad_fn=<ViewBackward0>)


Gradienten bezüglich der Eingangsgrößen berechnen (man kann den Gradienten nur bezüglich einer skalaren Ausgangsgröße berechnen)

In [62]:
test_output[0].backward(retain_graph=True)
print('Gradient des Ersten Ausgangselements bezüglich des Eingangsvektors: \n', test_input.grad)
test_output[1].backward()
print('Gradient des Zweiten Ausgangselements bezüglich des Eingangsvektors: \n', test_input.grad)

Gradient des Ersten Ausgangselements bezüglich des Eingangsvektors: 
 tensor([-0.2494,  0.2780,  0.4141])
Gradient des Zweiten Ausgangselements bezüglich des Eingangsvektors: 
 tensor([-0.5291,  0.4410,  0.5632])
